In [1]:
from scipy.optimize import linprog
import numpy as np
import pickle
import pandas as pd
import matplotlib.pyplot as plt
from gurobi import *

from dynamic_matching import DynamicMatchingBase
np.set_printoptions(precision=3)
pd.options.display.max_columns = None
from IPython.display import display, HTML
from IPython.display import display_html

### Define some utility functions

In [2]:
def in_constraint(v):
    if v[1]:
        return True
    else:
        return False

def print_tableau(matches):
    for i in range(matches.shape[2]):
        print("t=",i)
        print(matches[:,:,i])
        
def validate_allocation(allocations):
    agents_matched = allocations.sum(axis=2).sum(axis=0).sum()
    print("{} out of {} agents matched".format(agents_matched, J))
    print("{} agents have self matched".format(allocations[0,:,:].sum()))
    print("Self-matched agents are: ", np.nonzero(allocations[0,:,:].sum(axis=1))[0])

    assert agents_matched <= J, "An agent has been matched more than once"

    for i in range(I):
        for j in range(J):
            for t in range(T):
                maxit = min(T, t+d)
                assert allocations[i,j,t:maxit].sum() <= c[i], "A resource has been over allocated"

def create_tableau(I,J,T):
    valid_matches = np.zeros((I,J,T), np.int)

    #valid allocations based on arival time
    for t in range(T):
        valid_matches[:,max(0,t-d):min(t+1,J),t] = 1

    #make weights uniform across time
    pairing_weights = np.random.random(valid_matches.shape) 
    for t in range(1,pairing_weights.shape[2]):
        pairing_weights[:,:,t] = pairing_weights[:,:,0]
    
    pairing_weights[0,:,:] = 1e-5
    pairing_weights[valid_matches == 0] = -1

    return valid_matches, pairing_weights

def make_alpha_mapping(I,J,T,alphas,valid_matches):
    '''Creates an index into the alpha array for each position in valid matches'''
    constraints_d, _ = dual_constraint_matrix(valid_matches,pairing_weights,I,J,T)
    constraints_d = constraints_d[:,:alphas.size]
    alpha_map = np.zeros((*valid_matches.shape,constraints_d.shape[1]),dtype=np.bool)

    cix=0
    for i in range(I):
            for j in range(J):
                for t in range(T):
                    if valid_matches[i][j][t]:
                        alpha_map[i,j,t,:] = constraints_d[cix,:]
                        cix += 1
    return alpha_map
    

To make the dual constraint matrix: 
- Create a constraint map to see which alphas/betas apply at a given location in the primal. Each valid location will correspond to a constraint in the dual, and the variables == 1 will be those in the `cmap[i][j][t]`

In [3]:
def primal_constraint_matrix(valid_matches, I,J,T):

    constraints = np.zeros((T*I+J,valid_matches.size),dtype=np.float128)
    cix = 0
    #constraints limiting to one resource allocation in the time interval
    for i in range(I):
        for t in range(T):
            constraint = np.zeros((I,J,T), np.int)
            valid_mask = constraint.copy()
            endix = min(t+k[i],T)
            valid_mask[i,:,t:endix] = 1 
            constraint[(valid_matches == 1) & (valid_mask == 1)] = 1
            constraints[cix,:] = constraint.reshape((1, constraint.shape[0] * constraint.shape[1] * constraint.shape[2]))
            cix += 1

    #constraints limiting each agent to only match once            
    for j in range(J):
        constraint = np.zeros((I,J,T), np.int)
        valid_mask = constraint.copy()
        valid_mask[:,j,:] = 1

        constraint[(valid_matches == 1) & (valid_mask ==1)] = 1
        constraints[cix+j,:] = constraint.reshape((1, constraint.shape[0] * constraint.shape[1] * constraint.shape[2]))
    
    return constraints


def dual_constraint_matrix(valid_matches,pairing_weights,I,J,T):

    constraint_map = np.zeros((I,J,T,T*I+J), np.int)
    cix = 0

    #constraints limiting to one resource allocation in the time interval
    for i in range(I):
        for t in range(T):
            constraint = np.zeros((I,J,T), np.int)
            valid_mask = constraint.copy()

            endix = min(t+k[i],T)
            valid_mask[i,:,t:endix] = 1 
            constraint[(valid_matches == 1) & (valid_mask == 1)] = 1

            constraint_map[:,:,:,cix] = constraint.copy()
            cix += 1

    #constraints limiting each agent to only match once            
    for j in range(J):
        constraint = np.zeros((I,J,T), np.int)
        valid_mask = constraint.copy()
        valid_mask[:,j,:] = 1
        constraint[(valid_matches == 1) & (valid_mask ==1)] = 1
        constraint_map[:,:,:,cix] = constraint.copy()
        cix += 1

    dual_constraint_matrix = np.zeros((valid_matches.sum(), constraint_map.shape[3]))
    inequalities = np.zeros(valid_matches.sum())
    

    cix = 0
    for i in range(I):
        for j in range(J):
            for t in range(T):
                if valid_matches[i][j][t]:
                    dual_constraint_matrix[cix,:] = constraint_map[i,j,t,:] 
                    inequalities[cix] = pairing_weights[i,j,t]
                    cix += 1
    
    return dual_constraint_matrix, inequalities



In [4]:
def primal_solutions(pairing_weights, I, J, T):
    m = Model("dynamicmatch_primal")
    m.modelSense = GRB.MAXIMIZE
    m.setParam( 'OutputFlag', False )

    weights = pairing_weights.reshape(pairing_weights.shape[0] * pairing_weights.shape[1] * pairing_weights.shape[2])
    constraints = primal_constraint_matrix(valid_matches, I,J,T)


    keys = range(constraints.shape[1])
    variables = m.addVars(keys,
                    vtype=GRB.CONTINUOUS,
                     obj=weights,
                     name="primal",
                     lb=0)

    for cix, constraint in enumerate(constraints):
        equality = c[cix // T] if cix < T * I else 1
        m.addConstr(sum(variables[o]*c for o,c in filter(in_constraint, zip(variables,constraint))) <= equality)

    m.optimize()
    m.write('primal_formulation.lp')
    allocations = np.array([variables[var].X for var in variables], dtype=np.float128).reshape(pairing_weights.shape)

    return m.objVal, allocations


def dual_solutions(valid_matches, pairing_weights, I, J, T):
    md = Model("dynamicmatch_dual")
    md.modelSense = GRB.MINIMIZE
    md.setParam( 'OutputFlag', False )

    constraints_d, inequalities = dual_constraint_matrix(valid_matches,pairing_weights,I,J,T)
    c_d = np.ones(constraints_d.shape[1], dtype=np.float128)
    
    for ix in range(constraints_d.shape[1]):
        c_d[ix] = c[ix // T] if ix < T * I else 1
    
    keys = range(constraints_d.shape[1])
    variables = md.addVars(keys,
                    vtype=GRB.CONTINUOUS,
                    obj=c_d,
                    name="dual",
                    lb=0)

    for cix, constraint in enumerate(constraints_d):
        con = sum(variables[o]*c for o,c in filter(in_constraint, zip(variables,constraint))) >= inequalities[cix]
        md.addConstr(sum(variables[o]*c for o,c in filter(in_constraint, zip(variables,constraint))) >= inequalities[cix])

    md.optimize()
    duals = np.array([variables[var].X for var in variables],dtype=np.float128)
    betas = duals[duals.size - J:]
    alphas = duals[:duals.size - J]
    
    md.write('dual_formulation.lp')
    return md.objVal, alphas, betas


In [5]:
def make_alpha_tableau(valid_matches, alphas, I, T):

    alpha_tableau = np.zeros(valid_matches.shape)
    ii = 0
    it = 0
    for ax, alpha in enumerate(alphas):

        if ii < I:
            use_length = k[ax // T]
            alpha_tableau[ii,:,it:it+use_length] = alpha
            it = it+use_length

            if it >= alpha_tableau.shape[2]:
                it = 0
                ii += 1
                
    return alpha_tableau

def display_alphas(valid_matches, alphas, I, T):
    amap = make_alpha_mapping(I,J,T,alphas,valid_matches)
    alpha_viz = np.zeros(valid_matches.shape)

    for i in range(I):
        for j in range(J):
            for t in range(T):
                alpha_viz[i,j,t] = np.sum(alphas[amap[i,j,t]])
    return alpha_viz

def print_alpha_allocation_view(alphas, I,T):
    display(pd.DataFrame(alphas.reshape(I,T)))

### Online allocation alogrithm implementation
- At each time, for each candidate resource / utility pair, allocate if assignment term less than epsilon
- Track validity of candidate matches based on arrival/departure model, previously matched agents, and resource utilization times

In [6]:
def online_matching(I,J,T,k,c,alphas,betas,valid_matches,pairing_weights,epsilon=0):

    alpha_map = make_alpha_mapping(I,J,T,alphas,valid_matches)
    online_allocations = np.zeros(pairing_weights.shape)
    comps = np.zeros(pairing_weights.shape)
    utility = 0
    candidate_matches = valid_matches.copy()
    
    resource_uses = np.zeros((pairing_weights.shape[0],pairing_weights.shape[2]))

    for t in range(T):
        for j in range(J):
            for i in range(1,I):
                
#                 if i == 1 and j == 2 and t == 4:
#                     print('valid matches: ',valid_matches[i,j,t])
#                     print('candidate matches: ',candidate_matches[i,j,t])
#                     print('resource uses: ', resource_uses[i,t])
#                     print('ci', c[i])
                
                if candidate_matches[i,j,t] and resource_uses[i,t] < c[i]:
                    asum = np.sum(alphas[alpha_map[i,j,t] == 1])

                    comps[i,j,t] = np.abs(pairing_weights[i,j,t] - asum - betas[j])
                    #allocate if less than epsilon
                    if np.abs(pairing_weights[i,j,t] - asum - betas[j]) <= epsilon:
                        online_allocations[i,j,t] = 1
                        utility += pairing_weights[i,j,t]

                        #prevent matches with resource during time period and the same agent
                        candidate_matches[:,j,:] = 0
                        resource_uses[i,t:t+k[i]] += 1
        
            #agent hasn't been allocated, self match 
            if j == t - d and candidate_matches[0,j,t]:
                online_allocations[0,j,t] = 1
                utility += pairing_weights[0,j,t]
                candidate_matches[:,j,:] = 0
                resource_uses[0,t:t+k[i]] += 1

                    
    
    return utility, online_allocations,comps


### Utility functions for examining allocations

In [7]:
def debug_allocation_differences(online_allocs, offline_allocs, comps):
    diff = online_allocs - offline_allocs
    print("\nOnline allocated when it shouldn't (false positive)")
    for i in range(I):
        for j in range(J):
            for t in range(T):
                if diff[i,j,t] == 1 and i > 0:
                    print("Occurs at {},{},{}. Comparison value: {}".format(i,j,t, comps[i,j,t]))
    
    
    print("\nOnline didn't allocate when it should (false negative)")
    for i in range(I):
        for j in range(J):
            for t in range(T):
                if diff[i,j,t] == -1 and i > 0:
                    print("Occurs at {},{},{}. Comparison value: {}".format(i,j,t, comps[i,j,t]))
                    
def display_3D(tableaus_int, tableaus_float, names, c, k, T):
    '''Given an array of numpy arrays, transforms to dataframes and displays'''
    
    metadf = pd.DataFrame({
        'copies' :[c[i] for i in range(allocs.shape[0])],
        'use times' :[k[i] for i in range(allocs.shape[0])]
    })
    
    html_str = ('<th>' 
                + ''.join([f'<td style="text-align:center">{name}</td>' for name in names])
                + '</th>')

    int_ixs = len(tableaus_int)-1
    tableaus_int.extend(tableaus_float)
    
    for i in range(T):
        row_dfs = [pd.DataFrame(tableau[:,:,i]) for tableau in tableaus_int]
    
        html_str += ('<tr>' + "<td style='vertical-align:center'> t = {}".format(i) +
                     f"<td style='vertical-align:top'>{metadf.to_html(index=False, float_format='%i')}</td>" +
                     ''.join(f"<td style='vertical-align:top'> {df.to_html(index=False, float_format='%.3f') if ix > int_ixs else df.to_html(index=False,float_format='%i')}</td>"
                             for ix,df in enumerate(row_dfs)) + 
                     '</tr>')
        
    html_str = f'<table>{html_str}</table>'
    display_html(html_str, raw=True)
    
    
    

### Online matching pseudocode: 

- For time in T
    - For agent in agents
        - For resource in resources
            - if match is valid and resource is available
                - if `weight - alphas - beta <= epsilon`:
                    - allocate here
                    - update candidate matches and resource availability
        - if agent is about to leave and is not matched
            - self match here

### Reasons for descrpencies between online and primal solutions (see below for simulations)

- The online will only self match after `d` turns of not finding a match. The optimal primal may do it earlier. This doesn't impact utility. 
- The comparison term `w[i,j,t] - alphas - beta` is zero in places where online assignment should not occur (assigns when it shouldn't).
- The comparison term `w[i,j,t] - alphas - beta` is slightly greater than zero (~`1e-17`) in places where online assignment should occur (doesnt assign when it should). Making epsilon `1e-15` fixes this issue, but then leads to a few more false positive cases where the comparison is less than `1e-15`.

This could be a numerical problem, subtle issue with the implementation, or underlying issue with online assignment




### Define situation variables

In [24]:
J = 6 # number of agents
d = 2 # wait time for each agent

I = 3 # number of resources
k=np.array([1,9,5]) # use times
c=np.array([J,2,1]) # number of copies


T = J+d # total time for matches to occur

# valid_matches, pairing_weights = create_tableau(I,J,T)

### Testing online matching

In [28]:
objp, allocs = primal_solutions(pairing_weights, I, J, T)
objd, alphas, betas = dual_solutions(valid_matches,pairing_weights, I, J, T)
objo, online_allocs,comps = online_matching(I,J,T,k,c,alphas,betas,valid_matches,pairing_weights,epsilon=0)

print(objp)
print(objd)
print(objo)

2.4805353441426954
2.480535344142696
1.0118614247170965


### Visualizing online assignment

In [29]:
validate_allocation(allocs)
validate_allocation(online_allocs)
debug_allocation_differences(online_allocs, allocs, comps)

6.0 out of 6 agents matched
3.0 agents have self matched
Self-matched agents are:  [2 4 5]
6.0 out of 6 agents matched
4.0 agents have self matched
Self-matched agents are:  [2 3 4 5]

Online allocated when it shouldn't (false positive)
Occurs at 2,1,1. Comparison value: 0.0

Online didn't allocate when it should (false negative)
Occurs at 1,1,3. Comparison value: 0.0
Occurs at 2,3,4. Comparison value: 0.0


In [30]:
names=['','Primal allocations', 'Online allocations', 'Online assignment term', 'Pairing weights']
display_3D([allocs, online_allocs], [comps, pairing_weights], names, c, k, T)

Primal allocations Online allocations Online assignment term Pairing weights t = 0 
 
 
 copies 
 use times 
 
 
 
 
 6 
 1 
 
 
 2 
 9 
 
 
 1 
 5 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 1 
 0 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 1 
 0 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 
 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 
 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0.000 
 -1.000 
 -1.000 
 -1.000 
 -1.000 
 -1.000 
 
 
 0.858 
 -1.000 
 -1.000 
 -1.000 
 -1.000 
 -1.000 
 
 
 0.026 
 -1.000 
 -1.000 
 -1.000 
 -1.000 
 -1.000 
 
 
 t = 1 
 
 
 copies 
 use times 
 
 
 
 
 6 
 1 
 
 
 2 
 9 
 
 
 1 
 5 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 0 
 1 
 0 
 0 
 0 
 0 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 
 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 
 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0.000 
 0.000 
 -1.000 
 -1.000 
 -1.000 
 -1.000 
 
 
 0.858 
 0.627 
 -1.000 
 -1.000 
 -1.000 
 -1.000 
 
 
 0.026 
 0.154 
 -1.000 
 -1.000 
 -1.000 
 -1.000 
 
 
 t = 2 
 
 
 copies 
 use times 
 
 
 
 
 6 
 1 
 
 
 2 
 9 
 
 
 1 
 5 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 
 
 0.000 
 0.000 
 0.475 
 0.000 
 0.000 
 0.000 
 
 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0.000 
 0.000 
 0.000 
 -1.000 
 -1.000 
 -1.000 
 
 
 0.858 
 0.627 
 0.144 
 -1.000 
 -1.000 
 -1.000 
 
 
 0.026 
 0.154 
 0.008 
 -1.000 
 -1.000 
 -1.000 
 
 
 t = 3 
 
 
 copies 
 use times 
 
 
 
 
 6 
 1 
 
 
 2 
 9 
 
 
 1 
 5 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0 
 0 
 1 
 0 
 0 
 0 
 
 
 0 
 1 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 
 
 0.000 
 0.000 
 0.475 
 0.183 
 0.000 
 0.000 
 
 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 -1.000 
 0.000 
 0.000 
 0.000 
 -1.000 
 -1.000 
 
 
 -1.000 
 0.627 
 0.144 
 0.436 
 -1.000 
 -1.000 
 
 
 -1.000 
 0.154 
 0.008 
 0.995 
 -1.000 
 -1.000 
 
 
 t = 4 
 
 
 copies 
 use times 
 
 
 
 
 6 
 1 
 
 
 2 
 9 
 
 
 1 
 5 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 1 
 0 
 0 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0 
 0 
 1 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 
 
 0.000 
 0.000 
 0.475 
 0.183 
 0.277 
 0.000 
 
 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 -1.000 
 -1.000 
 0.000 
 0.000 
 0.000 
 -1.000 
 
 
 -1.000 
 -1.000 
 0.144 
 0.436 
 0.341 
 -1.000 
 
 
 -1.000 
 -1.000 
 0.008 
 0.995 
 0.850 
 -1.000 
 
 
 t = 5 
 
 
 copies 
 use times 
 
 
 
 
 6 
 1 
 
 
 2 
 9 
 
 
 1 
 5 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0 
 0 
 0 
 0 
 1 
 0 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0 
 0 
 0 
 1 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 
 


In [73]:
names=['','Primal allocations', 'sum of alphas', 'Pairing weights']
alpha_viz = display_alphas(valid_matches, alphas, I, T)

display_3D([allocs], [alpha_viz, pairing_weights], names, c, k, T)

Primal allocations sum of alphas Pairing weights t = 0 
 
 
 copies 
 use times 
 
 
 
 
 6 
 1 
 
 
 2 
 9 
 
 
 2 
 5 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 1 
 0 
 0 
 0 
 0 
 0 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 
 
 0.539 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 
 
 0.129 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0.000 
 -1.000 
 -1.000 
 -1.000 
 -1.000 
 -1.000 
 
 
 0.323 
 -1.000 
 -1.000 
 -1.000 
 -1.000 
 -1.000 
 
 
 0.129 
 -1.000 
 -1.000 
 -1.000 
 -1.000 
 -1.000 
 
 
 t = 1 
 
 
 copies 
 use times 
 
 
 
 
 6 
 1 
 
 
 2 
 9 
 
 
 2 
 5 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 
 
 0.539 
 0.539 
 0.000 
 0.000 
 0.000 
 0.000 
 
 
 0.129 
 0.129 
 0.000 
 0.000 
 0.000 
 0.000 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0.000 
 0.000 
 -1.000 
 -1.000 
 -1.000 
 -1.000 
 
 
 0.323 
 0.549 
 -1.000 
 -1.000 
 -1.000 
 -1.000 
 
 
 0.129 
 0.139 
 -1.000 
 -1.000 
 -1.000 
 -1.000 
 
 
 t = 2 
 
 
 copies 
 use times 
 
 
 
 
 6 
 1 
 
 
 2 
 9 
 
 
 2 
 5 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 1 
 0 
 0 
 0 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 
 
 0.539 
 0.539 
 0.539 
 0.000 
 0.000 
 0.000 
 
 
 0.129 
 0.129 
 0.129 
 0.000 
 0.000 
 0.000 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0.000 
 0.000 
 0.000 
 -1.000 
 -1.000 
 -1.000 
 
 
 0.323 
 0.549 
 0.684 
 -1.000 
 -1.000 
 -1.000 
 
 
 0.129 
 0.139 
 0.999 
 -1.000 
 -1.000 
 -1.000 
 
 
 t = 3 
 
 
 copies 
 use times 
 
 
 
 
 6 
 1 
 
 
 2 
 9 
 
 
 2 
 5 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 0 
 1 
 0 
 1 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 
 
 0.000 
 0.539 
 0.539 
 0.539 
 0.000 
 0.000 
 
 
 0.000 
 0.799 
 0.799 
 0.799 
 0.000 
 0.000 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 -1.000 
 0.000 
 0.000 
 0.000 
 -1.000 
 -1.000 
 
 
 -1.000 
 0.549 
 0.684 
 0.547 
 -1.000 
 -1.000 
 
 
 -1.000 
 0.139 
 0.999 
 0.665 
 -1.000 
 -1.000 
 
 
 t = 4 
 
 
 copies 
 use times 
 
 
 
 
 6 
 1 
 
 
 2 
 9 
 
 
 2 
 5 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 
 
 0.000 
 0.000 
 0.539 
 0.539 
 0.539 
 0.000 
 
 
 0.000 
 0.000 
 0.799 
 0.799 
 0.799 
 0.000 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 -1.000 
 -1.000 
 0.000 
 0.000 
 0.000 
 -1.000 
 
 
 -1.000 
 -1.000 
 0.684 
 0.547 
 0.007 
 -1.000 
 
 
 -1.000 
 -1.000 
 0.999 
 0.665 
 0.671 
 -1.000 
 
 
 t = 5 
 
 
 copies 
 use times 
 
 
 
 
 6 
 1 
 
 
 2 
 9 
 
 
 2 
 5 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 
 
 0.000 
 0.000 
 0.000 
 0.539 
 0.539 
 0.539 
 
 
 0.000 
 0.000 
 0.000 
 0.671 
 0.671 
 0.671 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 -1.000 
 -1.000 
 -1.000 
 0.000 
 0.000 
 0.000 
 
 
 -1.000 
 -1.000 
 -1.000 
 0.547 
 0.007 
 0.636 
 
 
 -1.000 
 -1.000 
 -1.000 
 0.665 
 0.671 
 0.922 
 
 
 t = 6 
 
 
 copies 
 use times 
 
 
 
 
 6 
 1 
 
 
 2 
 9 
 
 
 2 
 5 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 0 
 0 
 0 
 0 
 1 
 0 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 
 
 
 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 0.000 
 
 
 0.000 
 0.000 
 0.000 
 0.000 
 0.539 
 0.539 
 
 
 0.000 
 0.000 
 0.000 
 0

In [74]:
print_alpha_allocation_view(alphas, I,T)

,0,1,2,3,4,5,6,7
0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,0.539189,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,0.128501,0.0,0.0,0.670818,0.0,0.0,0.0,0.0


Need to modify below to be different based on 0 indicies

In [15]:
allocs.shape

(3, 6, 8)

In [19]:
padded.shape

(5, 8, 8)

In [21]:
padded = np.pad(allocs, 1, 'constant', constant_values=(-1))[1:-1,1:-1,:]

np.argmax(padded,axis=2)

array([[1, 1, 1, 1, 1, 1],
       [1, 1, 4, 6, 1, 1],
       [3, 2, 1, 1, 7, 8]])

In [18]:
padded = np.pad(allocs, 1, 'constant', constant_values=(-1))[:,:,1:-1]

### Now, set up some scenarios and see if matching follows our intuitions

In [80]:
J = 6 # number of agents
d = 2 # wait time for each agent

I = 3 # number of resources
k=np.array([1,9,5]) # use times
c=np.array([J,2,2]) # number of copies


T = J+d # total time for matches to occur

valid_matches, pairing_weights = create_tableau(I,J,T)

In [84]:
objp, allocs = primal_solutions(pairing_weights, I, J, T)
objd, alphas, betas = dual_solutions(valid_matches,pairing_weights, I, J, T)

print(objp)
print(objd)

3.9524460753594965
3.9524460753594965


In [105]:
print(pd.DataFrame(np.argmax(allocs,axis=1)))
display(pd.DataFrame(np.max(pairing_weights, axis=2)))

   0  1  2  3  4  5  6  7
0  0  0  0  0  0  0  0  0
1  0  0  0  1  0  0  4  0
2  0  0  2  0  0  3  0  5


,0,1,2,3,4,5
0,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010
1,0.478983,0.515323,0.118242,0.938233,0.856831,0.738432
2,0.496431,0.202180,0.295252,0.890856,0.036852,0.897752


In [ ]:
def display_3D(tableaus_int, tableaus_float, names, c, k, T):
    '''Given an array of numpy arrays, transforms to dataframes and displays'''
    
    metadf = pd.DataFrame({
        'copies' :[c[i] for i in range(allocs.shape[0])],
        'use times' :[k[i] for i in range(allocs.shape[0])]
    })
    
    html_str = ('<th>' 
                + ''.join([f'<td style="text-align:center">{name}</td>' for name in names])
                + '</th>')

    int_ixs = len(tableaus_int)-1
    tableaus_int.extend(tableaus_float)
    
    for i in range(T):
        row_dfs = [pd.DataFrame(tableau[:,:,i]) for tableau in tableaus_int]
    
        html_str += ('<tr>' + "<td style='vertical-align:center'> t = {}".format(i) +
                     f"<td style='vertical-align:top'>{metadf.to_html(index=False, float_format='%i')}</td>" +
                     ''.join(f"<td style='vertical-align:top'> {df.to_html(index=False, float_format='%.3f') if ix > int_ixs else df.to_html(index=False,float_format='%i')}</td>"
                             for ix,df in enumerate(row_dfs)) + 
                     '</tr>')
        
    html_str = f'<table>{html_str}</table>'
    display_html(html_str, raw=True)
    